In [1]:
import psycopg2
from math import sin, cos, asin, sqrt, radians

In [2]:
# Haversian Distance Function
# To compute distance between 2 points on earth given the co-ordiates of those points

def Haversian_distance(long1,lat1,long2,lat2):
    '''
    (long1,lat1) are the co-ordinates of the immediate previous point from the current location of the vessel
    (long2,lat2) are the co-ordinates of the immediate next point from the current location of the vessel
    '''
    long1,lat1,long2,lat2 = map(radians, [long1,lat1,long2,lat2])
    
    diff_long = long2-long1
    diff_lat = lat2-lat1

    i = sin(diff_lat/2)**2 + cos(lat1) * cos(lat2) * sin(diff_long/2)**2
    j = 2 * asin(sqrt(i))
    
    return j*6371  # 6371 is radius of earth in km
   

In [3]:
def shipInfoByMmsi(mmsi):
    try:
        conn = psycopg2.connect(
            dbname='shiptracking',
            user="postgres",
            password="zuzu",  # Replace securely
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()

        # Check if MMSI exists
        cursor.execute("SELECT EXISTS(SELECT 1 FROM ship_logs WHERE mmsi = %s);", (mmsi,))
        exists = cursor.fetchone()[0]

        if not exists:
            print(f"No data found for MMSI {mmsi}.")
            return []

        # Retrieve data
        cursor.execute("""
            SELECT mmsi, timestamp, payload, latitude, longitude
            FROM ship_logs
            WHERE mmsi = %s
            ORDER BY timestamp;
        """, (mmsi,))
        
        results = cursor.fetchall()
        logs = [
            {
                "mmsi": row[0],
                "timestamp": row[1],
                "payload": row[2],
                "latitude": row[3],
                "longitude": row[4]
            } for row in results
        ]

        return logs

    except Exception as e:
        print(f"Error: {e}")
        return []

    finally:
        cursor.close()
        conn.close()


In [4]:
logs = shipInfoByMmsi(29122001)

In [5]:
# To compute the total distance
source = (logs[0].get('longitude'), logs[0].get('latitude'))
dest = (logs[-1].get('longitude'), logs[-1].get('latitude'))

In [6]:
distance = Haversian_distance(source[0], source[1], dest[0], dest[1])
print(f"Total Distance covered: {round(distance, 2)} km")

Total Distance covered: 13392.08 km


In [7]:
# To get the  entire trajectory
print("Ship Trajectory: ")
print("Longitude  :  Latitude")
for i in range(len(logs)):
    print(f"{logs[i].get('longitude')}     {logs[i].get('latitude')}")

Ship Trajectory: 
Longitude  :  Latitude
79.646485     7.229672
79.612895     7.239448
79.579307     7.249227
79.545717     7.259003
79.512127     7.26878
79.478538     7.278557
79.444948     7.288335
79.411358     7.298112
79.37777     7.307888
79.34418     7.317665
79.31059     7.327443
79.277002     7.33722
79.243412     7.346997
79.209822     7.356773
79.176233     7.366552
79.142643     7.376328
79.109053     7.386105
79.075465     7.395882
79.041875     7.40566
79.008285     7.415437
78.974697     7.425213
78.941107     7.43499
78.907517     7.444768
78.873928     7.454545
78.840338     7.464322
78.806748     7.474098
78.77316     7.483877
78.73957     7.493653
78.705982     7.50343
78.672392     7.513207
78.638802     7.522985
78.605213     7.532762
78.571623     7.542538
78.538033     7.552315
78.504445     7.562093
78.470855     7.57187
78.437265     7.581647
78.403677     7.591423
78.370087     7.601202
78.336497     7.610978
78.302908     7.620755
78.269318     7.630532
78.2

In [8]:
""" Average Ship Speed is same as what it is assumed to compute the distances as dynamic ship speed depends on 
 various factors such as traffic near ports, wind directions, sea currents etc.."""

' Average Ship Speed is same as what it is assumed to compute the distances as dynamic ship speed depends on \n various factors such as traffic near ports, wind directions, sea currents etc..'